In [9]:
import pandas as pd
import glob
import os
from functools import reduce
import pyarrow

In [2]:
# dts_data = pd.read_csv("/content/drive/MyDrive/EC_Tower/station_metadata_len.csv")


file_path = '/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/prism_for_central_ameri/*.csv'



csv_files = glob.glob(file_path)



all_dataframes = []

for file in csv_files:
    # Read the first 11 rows as attributes
    attributes = pd.read_csv(file)

    all_dataframes.append(attributes)
    


# Start with the first DataFrame
result = all_dataframes[0]
result = result.drop('band', axis=1)
# Merge with the rest of the DataFrames
for df in all_dataframes[1:]:
    df = df.drop('band', axis=1)
    result = result.merge(df, on=['date','system:index','point_id','.geo'], how='outer')







In [3]:
dts_data = pd.read_csv("/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/station_metadata_len.csv")
df = dts_data[:][:161]
df
df['Latitude'] = df['Latitude'].round(4)
df['Longitude'] = df['Longitude'].round(4)

In [4]:
stations_id = {
  1 : [-119.58014102395923,36.4587205291358],
  2 : [-120.20100395913578,36.169668221548285],
  3 : [-120.10236864383359,36.94660861179753],
  4 : [-120.20264490832344,36.177667848838176],
  5 : [-120.20999796596058,36.83900158517578],
  6 : [-121.11800025437275,38.28899868843215],
  7 : [-121.49932919916024,38.099152461081694],
  8 : [-121.65310129898836,38.10872169194239],
  9 : [-121.64669802987558,38.11590084463843],
  10 : [-121.53500200758782,38.108998156207704],
  11 : [-121.64330019487288,38.10469958279218],
  12 : [-121.75470191649107,38.03689895127454],
  13 : [-122.02635927086914,38.2005568781381]
}

# Create separate dictionaries for latitude and longitude
lat_dict = {k: v[1] for k, v in stations_id.items()}
lon_dict = {k: v[0] for k, v in stations_id.items()}

# Add new columns
result['latitude'] = result['point_id'].map(lat_dict)
result['longitude'] = result['point_id'].map(lon_dict)


In [5]:
result['latitude'] = result['latitude'].round(4)
result['longitude'] = result['longitude'].round(4)


In [6]:
# Method 1: Using map()
ID_dict = dict(zip(df['Latitude'], df['Site ID']))
result['Site ID'] = result['latitude'].map(ID_dict)

General_classification_dict = dict(zip(df['Site ID'], df['General classification']))
result['General_classification'] = result['Site ID'].map(General_classification_dict)

result = result.rename(columns={'Site ID': 'Site_ID'})

Elevation_dict = dict(zip(df['Site ID'], df['Elevation (m)']))
result['Elevation'] = result['Site_ID'].map(Elevation_dict)

Energy_balance_dict = dict(zip(df['Site ID'], df['Energy balance']))
result['Energy_balance'] = result['Site_ID'].map(Energy_balance_dict)

Land_cover_details_dict = dict(zip(df['Site ID'], df['Land cover details']))
result['Land_cover_details'] = result['Site_ID'].map(Land_cover_details_dict)

Land_cover_type_dict = dict(zip(df['Site ID'], df['Land cover type']))
result['Land_cover_type'] = result['Site_ID'].map(Land_cover_type_dict)


result


,system:index,date,point_id,ppt,.geo,tdmean,tmean,vpdmax,vpdmin,tmin,tmax,latitude,longitude,Site_ID,General_classification,Elevation,Energy_balance,Land_cover_details,Land_cover_type
0,0_20090101,2008-12-31,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.717,5.503000,0.912,0.239,4.565,6.442,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards
1,0_20090102,2009-01-01,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.025,5.944000,1.700,0.494,4.836,7.053,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards
2,0_20090103,2009-01-02,1,2.898,"{""type"":""MultiPoint"",""coordinates"":[]}",4.454,6.111000,2.067,0.370,4.656,7.567,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards
3,0_20090104,2009-01-03,1,0.228,"{""type"":""MultiPoint"",""coordinates"":[]}",3.300,5.052000,3.851,0.202,0.441,9.665,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards
4,0_20090105,2009-01-04,1,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",1.732,4.622000,3.458,0.290,0.316,8.929,36.4587,-119.5801,MB_Pch,Croplands,90.0,1.00,Peach,Orchards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56974,12_20201227,2020-12-26,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",6.645,11.471399,8.840,0.435,6.817,16.126,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands
56975,12_20201228,2020-12-27,13,1.227,"{""type"":""MultiPoint"",""coordinates"":[]}",4.315,9.327499,7.957,0.464,4.567,14.088,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands
56976,12_20201229,2020-12-28,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",4.685,9.029900,6.475,0.323,4.429,13.631,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands
56977,12_20201230,2020-12-29,13,0.000,"{""type"":""MultiPoint"",""coordinates"":[]}",-1.101,9.277900,15.402,0.592,1.284,17.272,38.2006,-122.0264,US-Srr,Wetland/Riparian,8.0,0.86,Brackish tidal marsh,Wetlands


In [10]:

result.to_parquet("/home/mahdi/data/PRISM_tmin_stable_4kmD2_20160612_nc/prism_for_central_ameri/merged_prism_data_par_for_ameri_stations.parquet", engine="pyarrow")